In [63]:
import pandas as pd
from pulp import *

In [64]:
# Import Costs
manvar_costs = pd.read_excel('variable_costs.xlsx', index_col = 0)
manvar_costs

,USA,Germany,Japan,Brazil,India
Variable Costs ($/Unit),,,,,
USA,12,12,12,12,12
Germany,13,13,13,13,13
Japan,10,10,10,10,10
Brazil,8,8,8,8,8
India,5,5,5,5,5


In [65]:
# Import Costs
freight_costs = pd.read_excel('freight_costs.xlsx', index_col = 0)
freight_costs

,USA,Germany,Japan,Brazil,India
Freight Costs ($/Container),,,,,
USA,0,12250,1100,16100,8778
Germany,13335,0,8617,20244,10073
Japan,15400,22750,0,43610,14350
Brazil,16450,22050,28000,0,29750
India,13650,15400,24500,29400,0


In [66]:
# Variable Costs
var_cost = freight_costs/1000 + manvar_costs 

var_cost

,USA,Germany,Japan,Brazil,India
Freight Costs ($/Container),,,,,
USA,12.000,24.25,13.100,28.100,20.778
Germany,26.335,13.00,21.617,33.244,23.073
Japan,25.400,32.75,10.000,53.610,24.350
Brazil,24.450,30.05,36.000,8.000,37.750
India,18.650,20.40,29.500,34.400,5.000


In [67]:
# Import Costs (K$)
fixed_costs = pd.read_excel('fixed_cost.xlsx', index_col = 0)
fixed_costs

,Low,High
USA,6500,9500
Germany,4980,7270
Japan,6230,9100
Brazil,3230,4730
India,2110,6160


In [68]:
# Two types of plants: Low Capacity and High Capacity Plant
cap = pd.read_excel('capacity.xlsx', index_col = 0)
cap

,Low,High
Capacity (kUnits/year),,
USA,500,1500
Germany,500,1500
Japan,500,1500
Brazil,500,1500
India,500,3000


In [70]:
# -- Demand
demand = pd.read_excel('demand.xlsx', index_col=1)
demand.columns = [0,'Demand_1']
demand.drop(0,axis=1,inplace=True)
demand.drop('(Units/year)',axis=0,inplace=True)
growth_rate = {'USA': 0.10,
                'Germany': 0.08,
                'Japan': 0.07,
                'Brazil': 0.06,
                'India': 0.12}

no_of_years = 7

for i in range(2,no_of_years+2):
    l = 0
    demand[f'Demand_year_{i}'] = 0
    for (j,k) in growth_rate.items():
        demand.iloc[l,i-1] = demand.iloc[l,i-2]*(1+k)
        l = l + 1

demand

    

,Demand_1,Demand_year_2,Demand_year_3,Demand_year_4,Demand_year_5,Demand_year_6,Demand_year_7,Demand_year_8
USA,2800000,3080000.0,3388000.0,3726800.00,4.099480e+06,4.509428e+06,4.960371e+06,5.456408e+06
Germany,90000,97200.0,104976.0,113374.08,1.224440e+05,1.322395e+05,1.428187e+05,1.542442e+05
Japan,1700000,1819000.0,1946330.0,2082573.10,2.228353e+06,2.384338e+06,2.551242e+06,2.729829e+06
Brazil,145000,153700.0,162922.0,172697.32,1.830592e+05,1.940427e+05,2.056853e+05,2.180264e+05
India,160000,179200.0,200704.0,224788.48,2.517631e+05,2.819747e+05,3.158116e+05,3.537090e+05


In [71]:
# Define Decision Variables
loc = ['USA', 'Germany', 'Japan', 'Brazil', 'India']
size = ['Low', 'High']
# year = ['year_1','year_2','year_3','year_4','year_5','year_6','year_7','year_8']
year = list(demand.columns)

# Initialize Class
model = LpProblem("Capacitated Plant Location Model", LpMinimize)


# Create Decision Variables
x = LpVariable.dicts("production_", [(i,j,k) for i in loc for j in loc for k in year],
                     lowBound=0, upBound=None, cat='continuous')
y = LpVariable.dicts("plant_", 
                     [(i,s) for s in size for i in loc], cat='Binary')

# Define Objective Function
model += (lpSum([fixed_costs.loc[i,s] * y[(i,s)] * 1000 for k in year for s in size for i in loc])
          + lpSum([var_cost.loc[i,j] * x[(i,j,k)]   for i in loc for j in loc for k in year]))

# Add Constraints
for k in year:
    for j in loc:
        model += lpSum([x[(i, j, k)] for i in loc]) == demand.loc[j,k]

for k in year:
    for i in loc:
        model += lpSum([x[(i, j, k)] for j in loc]) <= lpSum([cap.loc[i,s]*y[(i,s)] * 1000
                                                       for s in size])


# Define logical constraint: Add a logical constraint so that if the high capacity plant in USA is open, then a low capacity plant in Germany is also opened.
# model += y[('USA','High_Cap')] <= y[('Germany','Low_Cap')]                                                       
                                                       
# Solve Model
model.solve()
print("Total Costs = {:,} ($/8 year)".format(int(value(model.objective))))
print('\n' + "Status: {}".format(LpStatus[model.status]))


# Dictionnary
dict_plant = {}
dict_prod = {}
for v in model.variables():
    if 'plant' in v.name:
        name = v.name.replace('plant__', '').replace('_', '').replace('_', '')
        dict_plant[name] = int(v.varValue)
        p_name = name
    else:
        name = v.name.replace('production__', '').replace('_', '').replace('_', '')
        dict_prod[name] = v.varValue
    print(name, "=", v.varValue)
    

Total Costs = 1,089,611,353 ($/8 year)

Status: Optimal
('Brazil','High') = 1.0
('Brazil','Low') = 0.0
('Germany','High') = 0.0
('Germany','Low') = 0.0
('India','High') = 1.0
('India','Low') = 1.0
('Japan','High') = 1.0
('Japan','Low') = 1.0
('USA','High') = 1.0
('USA','Low') = 1.0
('Brazil','Brazil','Demand1') = 145000.0
('Brazil','Brazil','Demandyear2') = 153700.0
('Brazil','Brazil','Demandyear3') = 162922.0
('Brazil','Brazil','Demandyear4') = 172697.32
('Brazil','Brazil','Demandyear5') = 183059.16
('Brazil','Brazil','Demandyear6') = 194042.71
('Brazil','Brazil','Demandyear7') = 205685.27
('Brazil','Brazil','Demandyear8') = 218026.39
('Brazil','Germany','Demand1') = 0.0
('Brazil','Germany','Demandyear2') = 0.0
('Brazil','Germany','Demandyear3') = 0.0
('Brazil','Germany','Demandyear4') = 0.0
('Brazil','Germany','Demandyear5') = 0.0
('Brazil','Germany','Demandyear6') = 0.0
('Brazil','Germany','Demandyear7') = 0.0
('Brazil','Germany','Demandyear8') = 0.0
('Brazil','India','Demand1') = 0

C:\Users\Mehul Singodia\AppData\Local\Programs\Python\Python310\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [72]:
# Capacity Plant
list_low, list_high = [], []
for l in loc:
    for cap in ['Low', 'High']:
        x = "('{}','{}')".format(l, cap)
        if cap == 'Low':
            list_low.append(dict_plant[x])
        else:
            list_high.append(dict_plant[x])
df_capacity = pd.DataFrame({'Location': loc, 'Low': list_low, 'High': list_high}).set_index('Location')
    
df_capacity

,Low,High
Location,,
USA,1,1
Germany,0,0
Japan,1,1
Brazil,0,1
India,1,1
